In [1]:
import cv2
import numpy as np

In [2]:
#Function to create point cloud file
def create_output(vertices, colors, filename):
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''.replace("  ","")
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')

In [3]:
# Load disparity map
disparity_map = np.load("../opencv-np-workshop/data/3d/disparity_map.npy")
K = np.load('../opencv-np-workshop/data/camera_params/K.npy')
img1 = cv2.imread("../opencv-np-workshop/data/3d/img1.png") # Used to get back some color info

In [4]:
K

array([[3.57130618e+03, 0.00000000e+00, 2.11505059e+03],
       [0.00000000e+00, 3.57886353e+03, 1.69787665e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [7]:
disparity_map.shape

(555, 641)

In [8]:
h, w = disparity_map.shape

#Heuristic for focal length. 
focal_length = 0.8*w #K[0,0]

#Perspective transformation matrix from the openCV documentation
Q = np.float32([[1,0,0,-w/2.0],
                [0,-1,0,h/2.0],
                [0,0,0,-focal_length],
                [0,0,-1,0]])

#Reproject points into 3D
points_3D = cv2.reprojectImageTo3D(disparity_map, Q)


In [9]:
#Get color points
colors = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

#Get rid of points with small values
mask_map = disparity_map > disparity_map.min()

#Mask colors and points. 
output_points = points_3D[mask_map]
output_colors = colors[mask_map]

#Define name for output file
output_file = '../opencv-np-workshop/data/3d/reconstructed.ply'

create_output(output_points, output_colors, output_file)

In [10]:
#Load focal length. 
focal_length = 0.8*w #K[0,0] if you do it properly!

h, w = disparity_map.shape

#Perspective transformation matrix from the OpenCV documentation
Q = np.float32([[1,0,0,-w/2.0],
                [0,-1,0,h/2.0],
                [0,0,0,-focal_length],
                [0,0,-1,0]])

#Reproject points into 3D
points_3D = cv2.reprojectImageTo3D(disparity_map, Q)


In [11]:
K[0][0]

3571.3061813656795

In [12]:
.8*w

512.8000000000001